In [1]:
import numpy as np

In [6]:
import pandas as pd
import pandas_profiling
import matplotlib.pylab as plt

In [3]:
topics = pd.read_csv(r"A:\PROGRAMMING\Test SD Solutions\topics.csv")
lists_topics_sample = pd.read_csv(r"A:\PROGRAMMING\Test SD Solutions\lists_topics_sample.csv")
sample_for_pred = pd.read_csv(r"A:\PROGRAMMING\Test SD Solutions\sample_for_pred.csv")

In [4]:
topics

,id,name
0,7,Electronics & Computers
1,8,"Toys, Kids & Baby"
2,5,"Beauty, Health & Grocery"
3,9,Automotive & Industrial
4,3,"Clothing, Shoes & Jewelry (Fashion)"
5,4,"Movies, Music & Games"
6,1,Books & Audible
7,2,"Home, Garden & Tools"
8,6,Sports & Outdoors


In [7]:
#lists_topics_sample.describe(include='all')
profile = pandas_profiling.ProfileReport(lists_topics_sample)
profile

Number of variables,3
Number of observations,1982
Total Missing (%),0.0%
Total size in memory,61.9 KiB
Average record size in memory,32.0 B
Numeric,2
Categorical,1
Date,0
Text (Unique),0
Rejected,0
Distinct count,1982


In [8]:
#sample_for_pred.describe(include = 'all')
profile = pandas_profiling.ProfileReport(sample_for_pred)
profile

Number of variables,2
Number of observations,485
Total Missing (%),0.0%
Total size in memory,11.4 KiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Date,0
Text (Unique),0
Rejected,0
Distinct count,422


In [7]:
set(lists_topics_sample.list_id).intersection(set(sample_for_pred.list_id))

{5619}

In [8]:
print(lists_topics_sample.list_id.nunique())
print(sample_for_pred.list_id.nunique())

1982
422


In [9]:
words = list()
for word in lists_topics_sample.list_name:
    words.extend(word.lower().split())

In [10]:
print(len(words))
print(len(set(words)))

4142
1843


In [11]:
words = list(set(words))

In [12]:
words_pred = list()
for word in sample_for_pred.list_name:
    words_pred.extend(word.lower().split())
words_pred = set(words_pred)
len(words_pred)

626

In [13]:
print(len(set(words).intersection(words_pred)))
common_words = list(set(words).intersection(words_pred))

354


In [14]:
#for f in common_words:
#    lists_topics_sample[f] = lists_topics_sample.list_name.apply(lambda x: f in x)

In [15]:
lists_topics_sample[:3]

,list_id,topic_id,list_name
0,5715,6,Inflatable Boats
1,4742,3,Anklet Bracelets
2,6655,2,Table Lamps


### read trained word vectors in .txt format to dictionary
#### loaded at http://nlp.stanford.edu/projects/glove/

In [16]:
def load_word2vec(file):
    word2vec = {} #skip information on first line
    fin= open(file, encoding="utf8")    
    for line in fin:
            items = line.replace('\r','').replace('\n','').split(' ')
            if len(items) < 10: continue
            word = items[0]
            vect = np.array([float(i) for i in items[1:] if len(i) > 1])
            word2vec[word] = vect
    return word2vec

word2vec_dict = load_word2vec(r'A:\PROGRAMMING\DATASETS\glove.6B.300d.txt')

#### words which were not found among loaded pre-trained words

In [17]:
def words_not_found_in_trained_dict(words):
    not_found = list()
    for word in words:
        if not word in word2vec_dict:
            not_found.append(word)
    print('Number:',len(not_found),'Percentage:',round(100.0*len(not_found)/len(words),2),
          "\nfirst 5 not found words in Word2Vec dictionary: ",not_found[:5])
    return not_found

not_found_train_words = words_not_found_in_trained_dict(words)
not_found_predict_words = words_not_found_in_trained_dict(words_pred)

Number: 67 Percentage: 3.64 
first 5 not found words in Word2Vec dictionary:  ['eyeliners', 'dethatchers', 'cambogia', 'k-cups', 'paracord']
Number: 32 Percentage: 5.11 
first 5 not found words in Word2Vec dictionary:  ['1tb', 'recievers', 'rollators', 'defuzzers', 'labret']


### build a function which allows to find the most similar words in word2vec_dict to those that were not found, and calculate the 300d vector for each of them

In [18]:
import difflib 
def find_closest(word):
    ## if plural:
    if word.endswith('ies'):
        word = word[:-3] + 'y'
    elif word.endswith('es'):
        word = word[:-2]
    elif word.endswith('s'):
        word = word[:-1]
    
    ## check if it helped:
    if word in word2vec_dict:
        return True, word, word2vec_dict[word]
    else:
        words_in_vect = list()
        for key in word2vec_dict:
            words_in_vect.append(key)
        ## use difflib library to search for similar words
        closest = difflib.get_close_matches(word, words_in_vect, n=2)
        ## return average of two vectors for two closest words
        return False, closest[0], list((np.array(word2vec_dict[closest[0]])+np.array(word2vec_dict[closest[1]]))/2)

In [19]:
not_found_train_words[-15:]

['yacon',
 'wet/dry',
 'filtrations',
 'putties',
 'pomades',
 'snoogle',
 'hvlp',
 'kegerators',
 'ruggeds',
 'daypacks',
 'rosehip',
 'monoculars',
 'refractometers',
 'under-sink',
 'boresighters']

In [20]:
for w in not_found_train_words[-15:]:
    plural, closest, vector = find_closest(w)
    print('Word:', w,'  Plural:',plural,'  Closest:',closest)

Word: yacon   Plural: False   Closest: acon
Word: wet/dry   Plural: False   Closest: wery
Word: filtrations   Plural: True   Closest: filtration
Word: putties   Plural: True   Closest: putty
Word: pomades   Plural: False   Closest: pomade
Word: snoogle   Plural: False   Closest: stooge
Word: hvlp   Plural: False   Closest: vlp
Word: kegerators   Plural: False   Closest: regenerator
Word: ruggeds   Plural: True   Closest: rugged
Word: daypacks   Plural: False   Closest: daypuck
Word: rosehip   Plural: False   Closest: roshi
Word: monoculars   Plural: True   Closest: monocular
Word: refractometers   Plural: True   Closest: refractometer
Word: under-sink   Plural: False   Closest: under-wing
Word: boresighters   Plural: False   Closest: boresight


## what's next

- now we have function "find_closest" which will allow to get 300d vectors for the words that were not found among pre-trained in word2vec_dict

- next we will calculate 300d vectors for each phrase in "words" and "words_pred" by averaging among words in the phrase

- last we will use cousine distance to find the closest vector

In [21]:
def get_phrase_vectors(phrase):
    words = phrase.lower().split()
    phrase_vec = np.zeros(300)
    for word in words:
        if word in word2vec_dict:
            phrase_vec = phrase_vec + word2vec_dict[word]
        else:
            sink, sink, vector = find_closest(word)
            phrase_vec = phrase_vec + vector
    phrase_vec = phrase_vec/len(words)
    return phrase_vec

In [22]:
def phrase_vectors_dict(data):
    phrase_vectors_dict = dict()
    for train_row_id in range(data.shape[0]):
        phrase = data.list_name[train_row_id]
        phrase_vectors_dict[train_row_id] = get_phrase_vectors(phrase)
    return phrase_vectors_dict

phrase_vectors_train = phrase_vectors_dict(lists_topics_sample)
phrase_vectors_pred = phrase_vectors_dict(sample_for_pred)

In [23]:
from scipy import spatial
def distance(list1, list2):
    return 1 - spatial.distance.cosine(list1, list2)

In [24]:
from operator import itemgetter
def check_predictions(pred_ix, k):
    print('Best predictions for ', sample_for_pred.list_name[pred_ix])
    distances = list()
    for train_ix in phrase_vectors_train:
        distances.append([train_ix, lists_topics_sample.list_name[train_ix], lists_topics_sample.topic_id[train_ix],
                          distance(phrase_vectors_train[train_ix], phrase_vectors_pred[pred_ix])])
    distances = sorted(distances, key=itemgetter(3), reverse = True)
    return distances[:k]

In [25]:
check_predictions(10, 5)

Best predictions for  Glass Desks


[[1581, 'Desks', 2, 0.75402125824679078],
 [1260, 'Glass Jars', 2, 0.72601509446800772],
 [1072, 'Corner Desks', 2, 0.6928085017187845],
 [955, 'Standing Desks', 2, 0.69210848992298535],
 [1451, 'Office Desks', 2, 0.67897339845045146]]

In [26]:
check_predictions(100, 5)

Best predictions for  Cfl Bulbs


[[348, 'Light Bulbs', 2, 0.66184728478891419],
 [700, 'Solar Lights', 6, 0.35091600092377728],
 [1049, 'Rechargeable Headlamps', 6, 0.34807166632619146],
 [560, 'Electric Stoves', 2, 0.32972049273252757],
 [1458, 'Ceramic Heaters', 2, 0.32745199513885215]]

In [27]:
check_predictions(400, 15)

Best predictions for  Address Labels


[[353, 'Label Makers', 2, 0.58163527563122153],
 [907, 'Label Printers', 7, 0.48382558630231665],
 [1546, 'Contact Solutions', 5, 0.42603150031588366],
 [1375, 'All In One Computers', 7, 0.39817146170331985],
 [1378, 'Credit Card Readers', 7, 0.39688669542546684],
 [211, 'Baby Food Makers', 2, 0.39448701503071981],
 [784, 'Electronic Game Calls', 6, 0.39264853422051216],
 [1369, 'Photo Albums', 1, 0.39212735827502432],
 [379, 'Sound Cards', 7, 0.38915840012125524],
 [203, 'Home Security Systems', 7, 0.38787164476413882],
 [610, 'Computer Speakers', 7, 0.38603654970338419],
 [608, 'Business Card Scanners', 7, 0.381838192435076],
 [1163, 'Card Readers', 7, 0.38010110826047161],
 [1213, 'Tackle Boxes', 6, 0.37903317678776682],
 [994, 'Sticker Makers', 7, 0.37629006924425767]]

In [28]:
check_predictions(90, 15)

Best predictions for  Baby Sound Machines


[[1198, 'Sleep Sound Machines', 5, 0.84797403415999384],
 [379, 'Sound Cards', 7, 0.72285080028372917],
 [902, 'Video Baby Monitors', 8, 0.67187946130182763],
 [996, 'Portable Washing Machines', 2, 0.66751863762780062],
 [923, 'Die Cutting Machines', 2, 0.65791857473394433],
 [260, 'Sound Bars', 7, 0.65332474813042229],
 [571, 'Bubble Machines', 8, 0.64874491281947955],
 [1610, 'Wireless Sound Bars', 7, 0.63307497236867505],
 [211, 'Baby Food Makers', 2, 0.63248990283817175],
 [1263, 'Baby Monitors', 8, 0.6273227427233552],
 [1381, 'Floor machines', 2, 0.62578673947145258],
 [167, 'Baby Books', 1, 0.61291121836215356],
 [1452, 'Baby Formulas', 8, 0.60622060006977485],
 [1701, 'Cigarette Rolling Machines', 2, 0.60277108099063104],
 [1425, 'Baby Bottles', 8, 0.59930137285766638]]

In [77]:
def predict_topics(pred_ix, k_closest):
    res_dict = dict()
    predictions = check_predictions(pred_ix, k_closest)
    for res in predictions:
        if res[3] < 0.22:
            continue
        if res[2] in res_dict:
            res_dict[res[2]] = res_dict[res[2]] + res[3]
        else:
            res_dict[res[2]] = res[3]
    print(res_dict,'\n')
    closest_key = max(res_dict, key=res_dict.get)
    closest_value = predictions[0][3]
    return closest_key, closest_value


In [78]:
topics_pred = list()
dist_pred = list()
for pred_ix in range(sample_for_pred.shape[0]):
    key, value = predict_topics(pred_ix, 15)
    topics_pred.append(key)
    dist_pred.append(value)

Best predictions for  Pocket Projectors
{2: 0.53006689353283709, 7: 7.7825366768783955} 

Best predictions for  Hard Drive Enclosures
{8: 0.51570887883784244, 9: 0.95973683704673951, 2: 1.0786450708662851, 7: 5.605584031786945} 

Best predictions for  Herb Grinders
{2: 5.9703077296338254, 5: 0.68274063725665246} 

Best predictions for  Womens Combat Boots
{3: 10.637091723153997, 6: 0.65289334890202377} 

Best predictions for  Keyboard Covers
{9: 1.6138715657992679, 2: 0.55752703793185299, 4: 1.155146023207718, 7: 5.3654473487632917} 

Best predictions for  Longboards
{2: 1.3510100306416835, 3: 0.62848411944942495, 4: 0.34402766864688117, 5: 0.66314449780465901, 6: 0.97420662367438549, 7: 0.67994888814487264, 9: 0.31974650296183316} 

Best predictions for  Enamel Paints
{9: 0.45255904788810564, 2: 6.6188028026900128, 5: 0.9906206682930867} 

Best predictions for  Bed Bug Sprays
{2: 6.7058502296750371, 5: 1.674404131589029} 

Best predictions for  Hemp Seeds
{9: 0.42850573945999559, 2: 0

In [31]:
list(sample_for_pred.list_name).index('Walking Canes')

328

In [32]:
sample_for_pred['topic_id'] = topics_pred

In [219]:
import matplotlib.pylab as plt
plt.hist(sample_for_pred.topic_id)
plt.show()

In [212]:
topics

,id,name
0,7,Electronics & Computers
1,8,"Toys, Kids & Baby"
2,5,"Beauty, Health & Grocery"
3,9,Automotive & Industrial
4,3,"Clothing, Shoes & Jewelry (Fashion)"
5,4,"Movies, Music & Games"
6,1,Books & Audible
7,2,"Home, Garden & Tools"
8,6,Sports & Outdoors


In [221]:
for i in range(1,10):
    print(i,' ',sample_for_pred[sample_for_pred.topic_id == i].shape[0])

1   1
2   247
3   40
4   10
5   26
6   29
7   105
8   20
9   7


In [222]:
lists_topics_sample[:4]

,list_id,topic_id,list_name
0,5715,6,Inflatable Boats
1,4742,3,Anklet Bracelets
2,6655,2,Table Lamps
3,5369,5,Eucalyptus Oils


In [223]:
sample_for_pred[:4]

,list_id,list_name,topic_id
0,6214,Pocket Projectors,7
1,5610,Hard Drive Enclosures,7
2,5640,Herb Grinders,2
3,6959,Womens Combat Boots,3


In [49]:
topics_name = list()
topics_id = list()
for ix in range(topics.shape[0]):
    topic_names = topics.name[ix].replace(',', '').replace('&', '').replace('(', '').replace(')', '').lower().split()    
    topic_ids = [topics['id'][ix]]*len(topic_names)
    topics_name.extend(topic_names)
    topics_id.extend(topic_ids)

In [74]:
def check_predictions_with_topic_words(pred_ix, k):
    print('Best predictions for ', sample_for_pred.list_name[pred_ix])
    distances = list()
    for topic_ix in range(len(topics_name)):
        distances.append([topic_ix, topics_name[topic_ix], topics_id[topic_ix],
                          distance(word2vec_dict[topics_name[topic_ix]], phrase_vectors_pred[pred_ix])])
    distances = sorted(distances, key=itemgetter(3), reverse = True)
    return distances[:k]

def predict_topics_with_topic_words(pred_ix, k_closest):
    res_dict = dict()
    predictions = check_predictions_with_topic_words(pred_ix, k_closest)
    for res in predictions:
        if res[2] in res_dict:
            res_dict[res[2]] = res_dict[res[2]] + res[3]
        else:
            res_dict[res[2]] = res[3]
    print(res_dict,'\n')
    closest_key = max(res_dict, key=res_dict.get)
    closest_value = predictions[0][3]
    return closest_key, closest_value

In [75]:
topics_pred2 = list()
dist_pred2 = list()
for pred_ix in range(sample_for_pred.shape[0]):
    key, value = predict_topics_with_topic_words(pred_ix, 3)
    topics_pred2.append(key)
    dist_pred2.append(value)

Best predictions for  Pocket Projectors
{1: 0.23854152534008333, 4: 0.27098062563983294, 7: 0.40090895762057632} 

Best predictions for  Hard Drive Enclosures
{2: 0.75518514578075058, 7: 0.37107423210382151} 

Best predictions for  Herb Grinders
{2: 0.22159206666916331, 5: 0.14493761807694461, 6: 0.17054193384956773} 

Best predictions for  Womens Combat Boots
{3: 0.94104926879305562, 4: 0.2339656523614505} 

Best predictions for  Keyboard Covers
{1: 0.29195963020755256, 4: 0.40614194917074609, 7: 0.34553208689694792} 

Best predictions for  Longboards
{8: 0.054343511754586515, 3: 0.050383600383539506, 6: 0.098544756710000403} 

Best predictions for  Enamel Paints
{3: 0.74623599133811735} 

Best predictions for  Bed Bug Sprays
{8: 0.28808030574621357, 2: 0.26307408825372403, 3: 0.28927233012240117} 

Best predictions for  Hemp Seeds
{2: 0.47269658306549978, 3: 0.25418014580180803} 

Best predictions for  Stilettos
{3: 0.70066945420488624} 

Best predictions for  Glass Desks
{3: 0.66119

In [62]:
sum(np.array(topics_pred) == np.array(topics_pred2))

202

In [117]:
#topics_pred_name = [topics.name[list(topics['id']).index(t)] for t in topics_pred]
#topics_pred_name2 = [topics.name[list(topics['id']).index(t)] for t in topics_pred2]
plt.hist(topics_pred2, bins=9, label = 'predictions with closest topic names')
plt.hist(topics_pred, bins=9, label = 'predictions with closest list names')
labels = ['Electronics', 'Toys, Kids', 'Beauty, Health', 'Automotive', 'Clothing, Jewerly', 
          'Movies, Mucis', 'Books', 'Home, Garden,', 'Sports']
plt.xticks(list(range(1,10)), labels, rotation='horizontal')
plt.legend(loc='upper right')
plt.title('Predicted topics')
plt.show()

In [116]:
plt.hist(dist_pred, bins=100, label = 'distanes of closest list names')
plt.hist(dist_pred2, bins=100, label = 'distances of closest topic names')
plt.legend(loc='upper right')
plt.title('Distribution of closest distances')
plt.show()

In [91]:
sample_for_pred.to_csv('A:\PROGRAMMING\Test SD Solutions\pred.csv')

In [105]:
list(topics['id']).index(9)

3

In [97]:
dir(topics.name[topics['id']])

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_prepare__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__int__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',